In [2]:
from underdog import info

## Overview
**Status**: Development

**Tested**: Windows only

**Requires**: Python 3.8+

The *underdog* package provides Python classes and functions to easily access free equity-related data via Panda DataFrames. Data is automatically downloaded, kept up to date, and cached on disk for convenience and to improve performance and limit API requests to the data providers (which often rate limit the APIs). Currently, data is pulled from [Polygon](https://polygon.io), [finviz](https://finviz.com), and [TD Ameritrade](https://www.tdameritrade.com/home.html).

Real-time streams of time and sales, level one quotes, and one minute candle data are also supported. The data streams are written to a persistent log and the data can be accessed from multiple processes.

The package is basically a convience wrapper over the underlying data services and the excellent [tda-api](https://tda-api.readthedocs.io/en/latest/index.html) package. 

Feel free to send questions, comments or feedback to: nanoonan at marvinsmind dot com.

## Installation

1. Install _underdog_ by opening a command prompt, navigating to the top-level directory of the git installation, and running the following command.

```
python -m pip install .
```

2. You'll need a free account with [Polygon](https://polygon.io) and a developer account with TD Ameritrade. The documentation for [tda-api](https://tda-api.readthedocs.io/en/latest/index.html) has more information on setting up a TDA developer account.

3. Finally, set up the environment variables used to access data from your accounts. The next cell describes these variables.

In [3]:
info('environment')

╒════════════════════════╤═══════════════════════════════════════════════╕
│ Environment Variable   │ Description                                   │
╞════════════════════════╪═══════════════════════════════════════════════╡
│ POLYGON_API_KEY        │ The API key from your Polygon account.        │
├────────────────────────┼───────────────────────────────────────────────┤
│ TDA_TOKEN_PATH         │ Path to file where TDA auth token is stored.  │
├────────────────────────┼───────────────────────────────────────────────┤
│ TDA_API_KEY            │ The API key from your TDA developer account.  │
├────────────────────────┼───────────────────────────────────────────────┤
│ TDA_REDIRECT_URI       │ In most cases this will be https://localhost. │
├────────────────────────┼───────────────────────────────────────────────┤
│ TDA_ACCOUNT_ID         │ Account id for your TDA trading account.      │
╘════════════════════════╧═══════════════════════════════════════════════╛


## API

In [4]:
# The main classes for accessing historic data
info('classes')

╒══════════════════════════╤════════════════════════════════════════════════════════════════════════╕
│ Type                     │ Description                                                            │
╞══════════════════════════╪════════════════════════════════════════════════════════════════════════╡
│ [Market, Day, IntraDay]  │ The historic data classes implement a Dict-like interface to access    │
│                          │ historic stock data. Valid keys are dates, integers, or strings        │
│                          │ containing dates in 'YYYY-MM-DD' format. Slices are also supported.    │
│                          │ Assuming x is an instance of Market, Day, or IntraDay, here are some   │
│                          │ examples: x['2020-01-01'], x['2020-01-01':'2021-01-01'], x[-252:]      │
│                          │ (returns last 252 trading days of data). Data is returned in a Panda   │
│                          │ DataFrame and downloaded and kept up to date automati

In [5]:
info('functions')

╒══════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════╕
│ Function                             │ Description                                                           │
╞══════════════════════════════════════╪═══════════════════════════════════════════════════════════════════════╡
│ intraday(symbol)                     │ Returns current intraday one-minute data for the specified ticker.    │
├──────────────────────────────────────┼───────────────────────────────────────────────────────────────────────┤
│ tickers(update = False)              │ Returns table of all tickers. Set update to True when calling for the │
│                                      │ first time or to download the latest data.                            │
├──────────────────────────────────────┼───────────────────────────────────────────────────────────────────────┤
│ ticker_details(symbol)               │ Returns info about a ticker like market cap, institutio

## Examples

In [6]:
from underdog import Market
market = Market()
# Invoke this once to download 2 years of data...will take over an hour
market.update()

True

In [7]:
# Get a dataframe of last two years of daily data for all tickers
market[:]

,symbol,volume,vwap,open,close,high,low,date,twap
0,A,1609122,77.1685,76.9800,77.4200,77.4600,76.3000,2019-04-26,76.86806
1,AA,2257704,26.9973,27.0500,26.9100,27.2500,26.8400,2019-04-26,27.046952
2,AAAU,30402,12.8574,12.8200,12.8450,12.8700,12.8200,2019-04-26,12.845833
3,AABA,3279169,76.1562,76.4400,76.1200,76.6800,75.5300,2019-04-26,76.101546
4,AAC,85690,1.5970,1.5900,1.6400,1.6600,1.5250,2019-04-26,1.591701
...,...,...,...,...,...,...,...,...,...
4624803,EMAG,72,20.8538,20.8700,20.8700,20.8700,20.8700,2021-05-21,EMAG
4624804,EMHC,132,30.4633,30.4700,30.4700,30.4700,30.4700,2021-05-21,EMHC
4624805,MXDU,164,38.9342,38.7804,38.7804,38.7804,38.7804,2021-05-21,MXDU
4624806,KCNY,12,35.2550,35.2850,35.2850,35.2850,35.2850,2021-05-21,KCNY


In [8]:
from underdog import ticker_details
# Get useful information about a ticker
ticker_details('TSLA')

{'market_cap': 559580000000,
 'shares_float': 774230000,
 'shares_outstanding': 963330000,
 'insider_ownership': 0.0,
 'institutional_ownership': 0.44,
 'retail_ownership': 0.56,
 'short_float': 0.05,
 'employees': 70757,
 'has_options': True,
 'is_shortable': True}

In [9]:
from underdog import IntraDay
# Last five days of TSLA intraday three minute data
df = IntraDay('TSLA', period = 3)[-5:]
# Filter for only regular hours trading (1 = pre-market, 2 = regular hours, 3 = extended hours)
df[df['trading_segment'] == 2]

,timestamp,open,close,low,high,volume,twap,trading_segment,timeslot
50,2021-05-17 09:30:00-04:00,575.8100,575.0975,573.3046,579.9900,618930,576.650699,2,110
51,2021-05-17 09:33:00-04:00,575.0000,582.5800,573.8000,582.8900,389324,578.251363,2,111
52,2021-05-17 09:36:00-04:00,582.7600,580.8200,580.5000,584.8000,313871,582.696109,2,112
53,2021-05-17 09:39:00-04:00,580.9361,578.6200,578.2900,584.0100,303696,581.208638,2,113
54,2021-05-17 09:42:00-04:00,578.8600,580.9650,578.8600,581.8000,188433,580.391372,2,114
...,...,...,...,...,...,...,...,...,...
1358,2021-05-21 15:45:00-04:00,581.7350,581.2376,580.0000,582.2700,240943,581.130732,2,235
1359,2021-05-21 15:48:00-04:00,581.1600,581.4200,580.6300,581.8200,143053,581.224215,2,236
1360,2021-05-21 15:51:00-04:00,581.2900,581.9000,581.0000,582.0000,152502,581.490256,2,237
1361,2021-05-21 15:54:00-04:00,581.8900,580.7800,580.7500,581.9064,186883,581.326165,2,238


In [10]:
from underdog import Day
# Last 20 years of daily XOM data
df = Day('XOM')[:]
df

,open,high,low,close,volume,date,twap
0,44.100,44.7200,44.050,44.530,9779400,2001-05-14,44.393036
1,44.530,44.7700,44.325,44.725,9016800,2001-05-15,44.543466
2,44.725,45.0000,44.520,44.750,16337000,2001-05-16,44.760015
3,44.575,44.5800,44.135,44.365,11597600,2001-05-17,44.350867
4,44.475,45.1250,44.400,45.100,14289200,2001-05-18,44.756795
...,...,...,...,...,...,...,...
10068,60.540,62.2500,60.430,62.190,25282521,2021-05-17,61.333535
10069,62.000,62.3000,60.400,60.430,25170801,2021-05-18,61.377788
10070,59.100,59.8550,58.415,58.980,28889805,2021-05-19,59.135165
10071,58.950,59.1500,58.240,58.840,20692495,2021-05-20,58.694267


## Stream Support (Experimental)

In [11]:
from underdog import (
    StreamReader,
    StreamType,
    StreamWriter
)

In [12]:
# Define a stream reader to handle each stream event. A reader
# runs as a background thread once started. Note: the reader
# does not need to run in the same process as the writer. 
# Multiple readers can read the same stream. Events are represented
# as tuples with event type, symbol, timestamps, and relevant price and size 
# info.
class MyReader(StreamReader):
    
    def on_event(self, event):
        print(event)
        
reader = MyReader('stockstream')
reader.start()

(2, 1621287113617, 'TSLA', 1621287000000, 573.38, 573.4, 573.49, 573.27, 697)
(2, 1621287113617, 'NIO', 1621287000000, 33.73, 33.72, 33.73, 33.72, 2701)
(1, 1621287118654, 'TSLA', 1621287095495, 573.1, 573.29, 1621287117722, 573.1, None)
(1, 1621287118654, 'NIO', 1621287114614, 33.72, 33.73, 1621287114614, 33.73, 1)
(1, 1621287123693, 'TSLA', 1621287121682, 573, None, 1621287121681, None, None)
(1, 1621287123693, 'NIO', 1621287120388, None, None, None, None, None)


In [13]:
# Start a stream writer for a set of symbols. In this example
# the stream includes time and sales, candle, and level 1 quote events
# for the symbols. The stream writer writes to a persistent
# log identified by the name passed to the constructor. One or
# more stream readers can read the events from multiple processes.
symbols = ['TSLA', 'NIO']
writer = StreamWriter(
    'stockstream', {
        StreamType.Chart: symbols,
        StreamType.Quote: symbols,
        StreamType.Trade: symbols
    },
    realtime = True
)
writer.start()

In [14]:
# Stop the writer thread.
writer.stop()